In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.mllib.tree import GradientBoostedTrees, GradientBoostedTreesModel
from sklearn.model_selection import train_test_split
from pyspark.sql.functions import *
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml import Pipeline
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.feature import VectorAssembler, MinMaxScaler
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report, precision_recall_curve, \
auc

In [2]:
config = pyspark.SparkConf().setAll([('spark.executor.memory', '8g'), ('spark.executor.cores', '4'), ('spark.cores.max', '4'), ('spark.driver.memory','8g'), ('spark.dynamicAllocation.enabled', 'false'), ("spark.driver.port", "7077")])

spark=SparkSession.builder.appName('Australian_rain_prediction').master("spark://10.147.17.13:7077").config(conf=config).getOrCreate()

sc = spark.sparkContext

print("spark session created")

22/12/03 14:11:37 WARN Utils: Your hostname, bigdata2022-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
22/12/03 14:11:37 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/12/03 14:11:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


spark session created


In [3]:
#df = spark.read.options(header = "true", inferschema = "true").csv('hdfs://localhost:9000/user/bigdata2022/datasets/big_data_project/australian_rain_dataset.csv')
df=spark.read.csv('hdfs://localhost:9000/user/bigdata2022/datasets/big_data_project/australian_rain_dataset.csv',inferSchema=True,header=True, sep=",")

ERROR:root:KeyboardInterrupt while sending command.                 (0 + 1) / 1]
Traceback (most recent call last):
  File "/home/bigdata2022/.local/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/bigdata2022/.local/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
print(df.head())

In [ ]:
print(df.columns)

In [ ]:
df_1 = df.drop('RainTomorrow')

In [ ]:
assembler = VectorAssembler(inputCols=df_1.columns, outputCol="features_to_scale")

In [ ]:
scaler = MinMaxScaler(inputCol="features_to_scale", outputCol="features_scaled")

In [ ]:
model = GBTClassifier(labelCol="RainTomorrow", featuresCol="features_scaled", maxIter=10, stepSize=0.005, maxDepth=8)

In [ ]:
pipeline = Pipeline(stages=[assembler, scaler, model])

In [ ]:
training_data, test_data = df.randomSplit([0.75, 0.25])

In [ ]:
#paramGrid = ParamGridBuilder().addGrid(model.stepSize, [0.01, 0.1, 0.5, 1.0])\
 #                             .addGrid(model.minWeightFractionPerNode, [0.01, 0.1, 0.25, 0.45])\
  #                            .addGrid(model.minWeightFractionPerNode, [0.01, 0.1, 0.25, 0.45])\
   #                           .addGrid(model.maxDepth, [3, 4, 5, 6, 7, 8, 9, 10, 11, 12])\
    #                          .addGrid(model.subsamplingRate, [0.01, 0.1, 0.5, 1.0]).build() 

In [ ]:
#crossval = CrossValidator(estimator=pipeline,
 #                         estimatorParamMaps=paramGrid,
  #                        evaluator=BinaryClassificationEvaluator(),
   #                       numFolds=5)

In [ ]:
#cvModel = crossval.fit(training_data)
pipeline_model = pipeline.fit(training_data)

In [ ]:
predictions = pipeline_model.transform(test_data)

In [ ]:
predictions.select("prediction", "RainTomorrow").show(100)

In [ ]:
t_test=predictions.select('RainTomorrow').toPandas()
t_hat=predictions.select('prediction').toPandas()

In [ ]:
print("\nAccuracy score on the test set: ", accuracy_score(t_test, t_hat))
print("\nF1-score score on the test set: ", f1_score(t_test, t_hat))